## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
import os
import mne
import eeg_toolkit
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


In [2]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data/'
epo_path = data_dir / time_win_path
stc_path = data_dir / "Source Time Courses (Nan Subs)"
EO_resting_save_path = stc_path / "Eyes Open"
EC_resting_save_path = stc_path / "Eyes Closed"
zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 

# Create save paths
[os.makedirs(path) for path in save_paths if not os.path.exists(path)]

[-2.5,0.0,2.5]


[]

In [3]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(processed_data_path,os.pardir),'sub_ids.txt')
# select only 64ch subs
sub_ids = sub_ids[7:]

# NAN_SUBS = ['018','021','C1.','C2.','C3.']
# sub_ids = [sub for sub in sub_ids if sub in NAN_SUBS]

print(sub_ids)

['018', '021', 'C1.', 'C2.', 'C3.']


In [4]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 2
Controls: 3
Total: 5


#### Look for subjects who do not have EO or EC cropped data

In [5]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

Subject: C1. missing eyes closed data
Subject: C1. missing eyes open data
['C1.']
['C1.']


In [6]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

In [10]:
nan_subjects=[]
for sub_id in sub_ids:  
    # Check if eyes open or eyes closed need to be computed
    EC_bool = True if sub_id not in no_eyes_open else False
    EO_bool = True if sub_id not in no_eyes_closed else False
    print(f"Subject: {sub_id} | EC: {EC_bool} | EO: {EO_bool}")
    
    # Skip if no Eyes Open
    if not EO_bool:
        continue
      
    # Compute source localization for subject and save 
    label_ts_All_Conds, sub_id_if_nan = source_localization.to_source(sub_id,
                                                                      processed_data_path,
                                                                        zscored_epochs_save_path,
                                                                        EC_resting_save_path,
                                                                        EO_resting_save_path,
                                                                        roi_names,
                                                                        times_tup,
                                                                        return_EO_resting = EO_bool,
                                                                        return_EC_resting = EC_bool,
                                                                        return_zepochs=True,#EO_bool, # cannot return epochs without eyes open noise segment
                                                                        average_dipoles=False,
                                                                        save_stc_mat=False,
                                                                        )
    utils.clear_display()
    # If sub_id(str) output, append to nan_subjects
    if sub_id_if_nan is not None:
        nan_subjects.append(sub_id_if_nan)
        
    # break

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 20485/20484 = 2.207463
    scale = 125765 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 2.6e-13 (2.2e-16 eps * 64 dim * 19  max singular value)
    Estimated rank (eeg): 60
    EEG: rank 60 computed from 64 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of w

### Assess results

In [11]:
label_ts_All_Conds

(None,
 None,
 [array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]),
  array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]),
  array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]),
  array([[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan

In [9]:
nan_subjects

['018']